In [ ]:
import os
#os.chdir('../../TCGA_SKCM/') #
#os.chdir('../../TCGA_KIRC/') #

In [ ]:
import yaml
import pandas as pd
import glob
import plotnine as p9
import numpy as np
from plotnine_prism import *

In [ ]:
out_folder = "out_benchmark"

In [ ]:
files = glob.glob(f"{out_folder}/prediction/*.yaml")
files

In [ ]:
results = []
for file in files:
    model = file.split("/")[-1].split("_")[0]
    with open(file) as stream:
        INFO = yaml.safe_load(stream)
    best_model = INFO["rank_test_score"][0] - 1
    
    mean_test_score = INFO["mean_test_score"][best_model]
    std_test_score = INFO["std_test_score"][best_model]
    mean_train_score = INFO["mean_train_score"][best_model]
    std_train_score = INFO["std_train_score"][best_model]

    
    results.append([model, mean_test_score, std_test_score, mean_train_score, std_train_score ])
results = pd.DataFrame(results, columns=["model", "mean_test_score", "std_test_score", "mean_train_score", "std_train_score"])
results

In [ ]:
results["model"] = pd.Categorical(results["model"], results.sort_values("mean_test_score", ascending=False)["model"])
((p9.ggplot(results, p9.aes("model", "mean_test_score"))) 
 + p9.geom_point()
 + p9.theme_bw()
 + p9.geom_errorbar(p9.aes(x="model", ymin="mean_test_score-std_test_score",
                           ymax="mean_test_score+std_test_score", color="model"), 
                    width=0.4, alpha=1, size=0.5)
 + scale_color_prism(palette = "colors")
 + p9.ylab("Balanced accuaracy")
)

In [ ]:
results.to_csv(f"{out_folder}/prediction/label_model_scores.csv", index=False)